INF8770 Technologies multimédias

Polytechnique Montréal

Exemple de codage arithmétique

In [1]:
import numpy as np

Message à coder. On codera seulement les SymbACoder premiers symboles. Limite des float. Sinon, on peut utiliser des bigfloat. 

In [2]:
Message = "ABAABCABACABBABCDAADAAABABAAABAABBABABAABAAB"
SymbACoder = 10

Calcul des probabilités de chaque symbole.

In [3]:
ProbSymb =[[Message[0], Message.count(Message[0])/len(Message)]]
nbsymboles = 1

for i in range(1,len(Message)):
    if not list(filter(lambda x: x[0] == Message[i], ProbSymb)):
        ProbSymb += [[Message[i], ProbSymb[-1][1]+Message.count(Message[i])/len(Message)]]
        nbsymboles += 1
        
longueurOriginale = np.ceil(np.log2(nbsymboles))*SymbACoder 

In [4]:
print(ProbSymb)

[['A', 0.5454545454545454], ['B', 0.8863636363636362], ['C', 0.9545454545454544], ['D', 0.9999999999999998]]


Codage du message. On divise successivement l'intervalle représentant le symbole à coder.

In [5]:
Code = ProbSymb[:]
Code = [['', 0]] + ProbSymb[:]
print(Code)
for i in range(8): 
    #Cherche dans quel intervalle est le symbole à coder
    temp = list(filter(lambda x: x[0] == Message[i], Code))
    indice = Code.index(temp[0])

    #Calcul des bornes pour coder le caractère
    Debut = Code[indice-1][1]
    Plage = Code[indice][1] - Debut
    print(Message[i], ' est dans l\'intervalle', indice, ' de ', Debut, ' à ', Debut + Plage)
    print()      
    #Nouveaux intervalles pour coder le prochain symbole
    Code = [['', Debut]]  
    for j in range(len(ProbSymb)):
        Code += [[ProbSymb[j][0], Debut+ProbSymb[j][1]*Plage]]
          
    print(Code)

[['', 0], ['A', 0.5454545454545454], ['B', 0.8863636363636362], ['C', 0.9545454545454544], ['D', 0.9999999999999998]]
A  est dans l'intervalle 1  de  0  à  0.5454545454545454

[['', 0], ['A', 0.29752066115702475], ['B', 0.4834710743801652], ['C', 0.5206611570247932], ['D', 0.5454545454545453]]
B  est dans l'intervalle 2  de  0.29752066115702475  à  0.4834710743801652

[['', 0.29752066115702475], ['A', 0.3989481592787377], ['B', 0.4623403456048083], ['C', 0.4750187828700224], ['D', 0.4834710743801651]]
A  est dans l'intervalle 1  de  0.29752066115702475  à  0.3989481592787377

[['', 0.29752066115702475], ['A', 0.35284475104159546], ['B', 0.38742230721945214], ['C', 0.39433781845502347], ['D', 0.3989481592787377]]
A  est dans l'intervalle 1  de  0.29752066115702475  à  0.35284475104159546

[['', 0.29752066115702475], ['A', 0.3276974374576997], ['B', 0.3465579226456215], ['C', 0.3503300196832059], ['D', 0.35284475104159546]]
B  est dans l'intervalle 2  de  0.3276974374576997  à  0.3465579

On choisit une valeur dans l'intervalle final. On cherche la valeur qui prend le moins de bits. On fait la somme de puissance négative de 2.

In [6]:
ok = True
valfinal = 0
p = 0
while ok:
    p += 1
    #Essayer différentes sommes de puissance négative de 2
    valfinal += np.power(2.0,-p)
    if valfinal > (Debut + Plage):
        valfinal -= np.power(2.0,-p) #Hors de la borne maximale, on annule l'ajout.
    elif valfinal > Debut :
        ok = False

print(valfinal)

0.344970703125


Conversion en bits pour évaluer la compression.

In [7]:
#Table hex vers binaire
hex2bin = dict('{:x} {:04b}'.format(x,x).split() for x in range(16))

def float_dec2bin(d):
    hx = float(d).hex() #Conversion float vers hex
    p = hx.index('p')
    #Conversion hex vers bin avec la table
    bn = ''.join(hex2bin.get(char, char) for char in hx[2:p])
    return ( bn.strip('0') + hx[p:p+2]
            + bin(int(hx[p+2:]))[2:])

messagecode = float_dec2bin(valfinal)
print(messagecode)

1.0110000101p-10


Longueur en bits du message codé et celle de l'original

In [8]:
longueur = messagecode.count('0') + messagecode.count('1') + messagecode.count('-') 
print("Longueur = {0}".format(longueur))
print("Longueur originale = {0}".format(longueurOriginale))

Longueur = 14
Longueur originale = 20.0
